In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils.engine as ue
import utils.visualizer as uv

In [2]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [3]:
plt.rcParams["animation.html"] = "jshtml"

In [4]:
data = pd.read_csv('Validation/full_data.csv')

In [5]:
with open("Validation/SHEBA_Data/12-levels.dat") as file:
    Z = [float(z.replace(' ', '')[:-1]) for z in file.readlines()]

## Инициализация профиля

In [8]:
data.columns

Index(['jday', 'uair', 'tair', 'qair', 'lwdo', 'swdo', 'prec', 'Focn_x',
       'albe', 'hs_best', 'hf_best', 'hi_best', 'hs_x', 'hi_x', 'hs_level',
       'hf_level', 'hi_level', 'Tss', 'Tis', 'Tib', 'Tsurf', 'hs_y', 'hi_y',
       'hf', 'Fcss', 'Fcsb', 'Fcis', 'Fcib', 'Fmib', 'Focn_y', 'Trod', 'Fcsf',
       'Tocn', 'focn_pit', 'T_1', 'T_2', 'T_3', 'T_4', 'T_5', 'T_6', 'T_7',
       'T_8', 'T_9', 'T_10', 'T_11', 'T_12', 'T_13', 'T_14', 'T_15', 'T_16',
       'T_17', 'T_18', 'T_19', 'T_20', 'T_21', 'T_22', 'T_23', 'T_24', 'T_25',
       'T_26', 'T_27', 'T_28', 'T_29', 'T_30', 'T_31', 'T_32', 'T_33', 'T_34',
       'T_35', 'T_36', 'T_37', 'T_38', 'T_39', 'T_40', 'T_41', 'T_42', 'T_43',
       'T_44', 'T_45', 'T_46', 'T_47', 'T_48', 'T_49', 'T_50', 'fsen', 'flat',
       'Tib_interp', 'Tis_interp'],
      dtype='object')

In [9]:
def get_init_from_data(data, dsigma_ice, dsigma_snow, snow_thickness_threshold=0.05, index=0):
    
    has_snow = (data.loc[index, 'hs_best'] - data.loc[index, 'hf_best']) >= snow_thickness_threshold
    
    filter_ice = (data.loc[index, 'hi_best'] < Z) & (Z < data.loc[index, 'hf_best'])
    filter_snow = (data.loc[index, 'hf_best'] < Z) & (Z < data.loc[index, 'hs_best'])

    Z_ice = np.concatenate((data.loc[index, ['hf_best']],
                            np.array(Z)[filter_ice],
                            data.loc[index, ['hi_best']]))

    Z_snow = np.concatenate((data.loc[index, ['hs_best']],
                            np.array(Z)[filter_snow],
                            data.loc[index, ['hf_best']]))

    T_ice = np.concatenate((data.loc[index, ['Tis_interp']],
                            data.loc[index, 'T_1':'T_50'][filter_ice],
                            data.loc[index, ['Tib_interp']]))

    T_snow = np.concatenate((data.loc[index, ['Tss']],
                             data.loc[index, 'T_1':'T_50'][filter_snow],
                             data.loc[index, ['Tis_interp']]))

    sigma_ice_nodes = np.concatenate(([0.0], dsigma_ice.cumsum()))
    sigma_ice_centers = (sigma_ice_nodes[:-1] + sigma_ice_nodes[1:])/2
    Z_ice_new = Z_ice[-1] + sigma_ice_centers*(Z_ice[0] - Z_ice[-1])

    sigma_snow_nodes = np.concatenate(([0.0], dsigma_snow.cumsum()))
    sigma_snow_centers = (sigma_snow_nodes[:-1] + sigma_snow_nodes[1:])/2
    Z_snow_new = Z_snow[-1] + sigma_snow_centers*(Z_snow[0] - Z_snow[-1])

    T_ice_new = np.interp(Z_ice_new, Z_ice[::-1], T_ice[::-1])
    T_snow_new = np.interp(Z_snow_new, Z_snow[::-1], T_snow[::-1])
    
    return T_ice_new, T_snow_new, \
            data.loc[index, 'Tib_interp'], data.loc[index, 'Tis_interp'], \
            (data.loc[index, 'Tss'] if has_snow else np.nan), \
            dsigma_ice*(data.loc[index, 'hf_best'] - data.loc[index, 'hi_best']), \
            (1.0 if has_snow else np.nan)*dsigma_snow \
            *(data.loc[index, 'hs_best'] - data.loc[index, 'hf_best'])

In [7]:
dsigma_ice = np.full(20, 1.0/20)
dsigma_snow = np.full(5, 1.0/5)

In [11]:
T_ice_init, T_snow_init, Toi_init, Tis_init, Tsa_init, \
dzi_init, dzs_init = get_init_from_data(data, dsigma_ice, dsigma_snow)

In [39]:
def find_index(time_sec):
    return np.searchsorted((data['jday'] - data.loc[0, 'jday'])*24*3600, time_sec) - 1

def find_index_fast(time_sec, seconds):
    return np.searchsorted(seconds, time_sec) - 1

In [43]:
test = ((data['jday'] - data.loc[0, 'jday'])*24*3600).values
%timeit find_index(3600*1200 + 1000)
%timeit find_index_fast(3600*1200 + 1000, test)

251 µs ± 5.64 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.15 µs ± 299 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
e_s = lambda T: 6.11*np.exp(ue.c1_i*T/(T + ue.T0 - ue.c2_i))
q_surf = lambda T: 0.622*e_s(T)/(ue.P_surf - 0.378*e_s(T))

albedo = lambda time: data.loc[find_index(time), 'albe']
T_a = lambda time: data.loc[find_index(time), 'tair'] - ue.T0
u_a = lambda time: data.loc[find_index(time), 'uair']
q_a = lambda time: data.loc[find_index(time), 'qair'] * 1e-3
p = lambda time: data.loc[find_index(time), 'prec'] * ue.p_ms
F_sw = lambda time: data.loc[find_index(time), 'swdo']
F_lw = lambda time: data.loc[find_index(time), 'lwdo']
F_bolz = lambda T: -ue.sigma*(T + ue.T0)**4
F_sh = lambda T, time: ue.rho_a*ue.c_pa*ue.C_sh*u_a(time)*(T_a(time) - T)
F_lh = lambda T, time: ue.rho_a*ue.L_s0*ue.C_lh*u_a(time)*(q_a(time) - q_surf(T))
F_P = lambda T, time: p(time)*ue.rho_w*ue.c_pw*max(T_a(time) - T, 0)
F_atm = lambda T, time, i_0: ue.emissivity*(F_lw(time) + F_bolz(T)) \
                           + (1 - albedo(time))*(1 - i_0)*F_sw(time) \
                           + F_sh(T, time) + F_P(T, time) + F_lh(T, time) 

F_atm_ice = lambda T, time: F_atm(T, time, ue.i0_i)
F_atm_snow = lambda T, time: F_atm(T, time, ue.i0_s)
F_ocn = lambda T, time: data.loc[find_index(time), 'focn_pit']

In [37]:
%%time
sheba_proc = ue.main_process \
(
    time_step=3600.0,
    time_end=5*3600.0,
    N_pseudoiter=50,
    Ti_init=T_ice_init,
    Ts_init=T_snow_init,
    Toi_init=Toi_init,
    Tis_init=Tis_init,
    Tsa_init=Tsa_init,
    dzi_init=dzi_init,
    dzs_init=dzs_init,
    salinity=np.linspace(4.0, 1.0, len(dzi_init)),
    snow_thickness_threshold=0.01,
    Ta=T_a,
    p=p,
    F_atm_ice=F_atm_ice,
    F_atm_snow=F_atm_snow,
    F_sw=F_sw,
    F_ocn=F_ocn
)

Time 1.0 h.: Snow-ice freezing...
Time 2.0 h.: Snow-ice freezing...
Time 3.0 h.: Snow-ice freezing...
Time 4.0 h.: Snow-ice freezing...
Time 5.0 h.: Snow-ice freezing...
CPU times: user 12.4 s, sys: 28 ms, total: 12.5 s
Wall time: 12.5 s


In [44]:
e_s = lambda T: 6.11*np.exp(ue.c1_i*T/(T + ue.T0 - ue.c2_i))
q_surf = lambda T: 0.622*e_s(T)/(ue.P_surf - 0.378*e_s(T))

albe = data['albe'].values
tair = data['tair'].values
uair = data['uair'].values
qair = data['qair'].values
prec = data['prec'].values
swdo = data['swdo'].values
lwdo = data['lwdo'].values
focn_pit = data['focn_pit'].values

albedo = lambda time: albe[find_index_fast(time, test)]
T_a = lambda time: tair[find_index_fast(time, test)] - ue.T0
u_a = lambda time: uair[find_index_fast(time, test)]
q_a = lambda time: qair[find_index_fast(time, test)] * 1e-3
p = lambda time: prec[find_index_fast(time, test)] * ue.p_ms
F_sw = lambda time: swdo[find_index_fast(time, test)]
F_lw = lambda time: lwdo[find_index_fast(time, test)]
F_bolz = lambda T: -ue.sigma*(T + ue.T0)**4
F_sh = lambda T, time: ue.rho_a*ue.c_pa*ue.C_sh*u_a(time)*(T_a(time) - T)
F_lh = lambda T, time: ue.rho_a*ue.L_s0*ue.C_lh*u_a(time)*(q_a(time) - q_surf(T))
F_P = lambda T, time: p(time)*ue.rho_w*ue.c_pw*max(T_a(time) - T, 0)
F_atm = lambda T, time, i_0: ue.emissivity*(F_lw(time) + F_bolz(T)) \
                           + (1 - albedo(time))*(1 - i_0)*F_sw(time) \
                           + F_sh(T, time) + F_P(T, time) + F_lh(T, time) 

F_atm_ice = lambda T, time: F_atm(T, time, ue.i0_i)
F_atm_snow = lambda T, time: F_atm(T, time, ue.i0_s)
F_ocn = lambda T, time: focn_pit[find_index_fast(time, test)]

In [46]:
%%time
sheba_proc = ue.main_process \
(
    time_step=3600.0,
    time_end=5*3600.0,
    N_pseudoiter=50,
    Ti_init=T_ice_init,
    Ts_init=T_snow_init,
    Toi_init=Toi_init,
    Tis_init=Tis_init,
    Tsa_init=Tsa_init,
    dzi_init=dzi_init,
    dzs_init=dzs_init,
    salinity=np.linspace(4.0, 1.0, len(dzi_init)),
    snow_thickness_threshold=0.01, 
    Ta=T_a,
    p=p,
    F_atm_ice=F_atm_ice,
    F_atm_snow=F_atm_snow,
    F_sw=F_sw,
    F_ocn=F_ocn
)

Time 1.0 h.: Snow-ice freezing...
Time 2.0 h.: Snow-ice freezing...
Time 3.0 h.: Snow-ice freezing...
Time 4.0 h.: Snow-ice freezing...
Time 5.0 h.: Snow-ice freezing...
CPU times: user 460 ms, sys: 0 ns, total: 460 ms
Wall time: 463 ms


In [ ]:
uv.animate([sheba_proc])

In [ ]:
data_proc = ue.process_from_data(Z,
                                 data.loc[:500, 'T_1':'T_50'].values, data.loc[:500, 'Tss'].values,
                                 data.loc[:500, 'Tis'].values, data.loc[:500, 'hi_best'].values,
                                 data.loc[:500, 'hf_best'].values, data.loc[:500, 'hs_best'].values,
                                 np.ones(20)/20.0, np.ones(5)/5.0,
                                 (data.loc[:500, 'jday'] - data.loc[0, 'jday']).values*24.0*3600.0, ue.rho_i)

In [ ]:
uv.animate([sheba_proc, data_proc])